In [1]:
# https://ieeexplore.ieee.org/document/9627980 (Paper URL)

## COMMANDS TO RUN ##
# !git clone https://github.com/airtlab/A-Dataset-for-Automatic-Violence-Detection-in-Videos
# !mv /content/A-Dataset-for-Automatic-Violence-Detection-in-Videos /content/drive/MyDrive/

# Violence Detection using AirtLab Dataset

## Preparations and setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
print("Importing Libraries...")
!pip install av -q
## Numerical Libraries ##
import numpy as np
import pandas as pd

## Visualization Libraries ##
import matplotlib.pyplot as plt

## Random ##
import random

## PyTorch Modules ##
import torch
from torch import nn, optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset, DataLoader, random_split

## TorchVision Modules ##
from torchvision.io import VideoReader
from torchvision.transforms.transforms import Resize

## TorchCodec Modules ##
try:
    from torchcodec.decoders import VideoDecoder
except ModuleNotFoundError:
    print("  Can't find torchcodec, installing...")
    !pip install torchcodec av -q
    from torchcodec.decoders import VideoDecoder
    print("    Successfully installed torchcodec!")

## TorchMetrics Modules ##
try:
    from torchmetrics import Accuracy, F1Score, Precision, Recall
except ModuleNotFoundError:
    print("  Can't find torchmetrics, installing...")
    !pip install torchmetrics -q
    from torchmetrics import Accuracy, F1Score, Precision, Recall
    print("    Successfully installed torchmetrics!")

## Typing library for type hints ##
from typing import List

## Utility libraries ##
import os
import warnings
from tqdm.auto import tqdm

## Ignore Warnings ##
warnings.filterwarnings("ignore")

## Initialize TensorBoard for experiment tracking ##
print("  Initializing SummaryWriter..")
writer = SummaryWriter(
    "./runs/"
)
print("Done!")

Importing Libraries...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 22.6 MB/s eta 0:00:00
  Can't find torchcodec, installing...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.9 MB/s eta 0:00:00
  Can't find torchmetrics, installing...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 15.3 MB/s eta 0:00:00
  Initializing SummaryWriter..
Done!


In [4]:
# Device Agnostic
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Utility Functions

In [5]:
def print_sizes(model, input_tensor):
    output = input_tensor
    for m in model.children():
        output = m(output)
        print(f"Layer: {m}\n{output.shape}\n-------------------------------")
    return output

## Dataset Structure

In [6]:
class ViolenceDetectionDataset(Dataset):
    """
    Builds the dataset with a lazy-loading approach to avoid huge memory "leaks" (eager loading for this dataset will consume huge amounts of memory, although not memory leak in the traditioal sense, still consumes lots of memory.)

    Parameters:
    video_paths (List[str]); List of paths to access videos
    labels (List[int]): List of labels
    normalize (bool, default False): Whether to normalize the returned frames or not
    """
    def __init__(self, video_paths:List[str], labels:List[int], normalize:bool=False):
        # self.cam1_vid_paths = cam1_vid_paths
        # self.cam2_vid_paths = cam2_vid_paths
        self.video_paths = video_paths
        self.labels = labels
        self.normalize = normalize

    @staticmethod
    def _read_video(video_path:str, target_num_frames:int=16, frame_rate:int=30, normalize:bool=False) -> List[torch.Tensor]:
        """
        Reads video file

        Parameters:
        video_path (str): The path to the video (can be the relative or absolute path)
        target_num_frames (int, default 16): The number of sample frames to extract from the video
        frame_rate (int, default 30, DO NOT MODIFY IF USING AIRTLAB DATASET): the videos frame rate (fps)
        normalize (bool, defualt False): Whether to normalize the sampled frames or not

        Returns:
        List[torch.Tensor()]: A list of tensors representing the extracted sample frames from the video
        """

        frames = []
        resizer = Resize((112, 112))
        video_reader = VideoReader(video_path, 'video')

        # Read the video frame by frame
        for frame in video_reader:
            frames.append(frame['data'])

        # Extract information about the video
        num_frames = len(frames)
        num_seconds = num_frames/frame_rate # Get the length of the video in seconds
        sampling_step = round((frame_rate*num_seconds)/target_num_frames) # Get the number of frames to sample per each second

        # Sample the video
        sampled_frames = frames[::sampling_step]
        number_of_samples = len(sampled_frames)
        if number_of_samples > target_num_frames: # Sometimes it can take one extra frame making the total number of frames target_num_frames+1
            sampled_frames = sampled_frames[:target_num_frames]

        elif number_of_samples < target_num_frames: # If the number of frames is less than target_num_frames, then duplicate a random frame
            diff = target_num_frames - number_of_samples # Number of frames to duplicate
            mid_frame = (number_of_samples//2) # The middle frame, most likely captures an action, so duplicating that (or the one before/after it) will at least be a meaningful duplication.
            i = random.choice([mid_frame-1, mid_frame, mid_frame+1]) # Random frame index
            frame_to_duplicate = sampled_frames[i]
            sampled_frames.extend([frame_to_duplicate] * diff)

        # resize the sampled frames (better than resizing all frames for computational reasons)
        frames = []
        if normalize:
            for sample in sampled_frames:
                processed_frame = resizer(sample.float()/255.0) # Normalize each frame to be in range [0,1]
                frames.append(processed_frame)
        else:
            for sample in sampled_frames:
                processed_frame = resizer(sample.float())
                frames.append(processed_frame)
        return torch.stack(frames).permute(1, 0, 2, 3).to(device)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # cam1_vid = self._read_video(video_path=self.cam1_vid_paths[idx], normalize=self.normalize)
        # cam2_vid = self._read_video(video_path=self.cam2_vid_paths[idx], normalize=self.normalize)
        X = self._read_video(video_path=self.video_paths[idx], normalize=self.normalize)
        y = torch.Tensor([self.labels[idx]]).to(device)
        return X, y

## Read Videos functions

In [7]:
# Read Videos
def read_nonViolent_videos() -> List[torch.Tensor] | List[torch.Tensor] | List[int]:
    """
    Read the Non-Violent video paths. Mainly for lazy loading in a later Dataset class

    Returns:
    cam1_vids: Video paths from camera 1
    cam2_vids: Video paths from camera 2
    label: a List of the labels (0s in this case, signifying non-violent)
    """
    nonViolent_cam1_path = "/content/drive/MyDrive/Datasets/A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/non-violent/cam1"
    nonViolent_cam2_path = "/content/drive/MyDrive/Datasets/A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/non-violent/cam2"
    num_nonViolent_videos = 60
    cam1_vid_paths = []
    cam2_vid_paths = []
    for video_num in range(1, num_nonViolent_videos+1):
        # Prepare the relative path of the video (for both cameras)
        video_index = f"{video_num}.mp4"
        cam1_video_path = os.path.join(nonViolent_cam1_path,video_index)
        cam2_video_path = os.path.join(nonViolent_cam2_path,video_index)

        # Append video to list of videos
        cam1_vid_paths.append(cam1_video_path)
        cam2_vid_paths.append(cam2_video_path)
    label = [0]*num_nonViolent_videos*2 # 0 is Non Violent class
    return cam1_vid_paths, cam2_vid_paths, label

def read_violent_videos() -> List[torch.Tensor] | List[torch.Tensor] | List[int]:
    """
    Read the Violent video paths. Mainly for lazy loading in a later Dataset class

    Returns:
    cam1_vids: Video paths from camera 1
    cam2_vids: Video paths from camera 2
    label: a List of the labels (1s in this case, signifying violent)
    """
    violent_cam1_path = "/content/drive/MyDrive/Datasets/A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/violent/cam1"
    violent_cam2_path = "/content/drive/MyDrive/Datasets/A-Dataset-for-Automatic-Violence-Detection-in-Videos/violence-detection-dataset/violent/cam2"
    num_violent_videos = 115
    cam1_vid_paths = []
    cam2_vid_paths = []
    for video_num in range(1, num_violent_videos+1):
        # Prepare the relative path of the video (for both cameras)
        video_index = f"{video_num}.mp4"
        cam1_video_path = os.path.join(violent_cam1_path,video_index)
        cam2_video_path = os.path.join(violent_cam2_path,video_index)

        # Append video to list of videos
        cam1_vid_paths.append(cam1_video_path)
        cam2_vid_paths.append(cam2_video_path)
    label = [1]*num_violent_videos*2 # 1 is Violent class
    return cam1_vid_paths, cam2_vid_paths, label

def read_dataset() -> List[torch.Tensor] | List[torch.Tensor] | List[int]:
    """
    Read the full dataset video paths. Mainly for lazy loading in a later Dataset class

    Returns:
    cam1_vids: Video paths from camera 1
    cam2_vids: Video paths from camera 2
    label: a List of the labels. Where each label index corrosponds to a video (first label is the first video's label, and so on...)
    """
    print("Loading Non-Violent Dataset...")
    nonv_cam1_vid_paths, nonv_cam2_vid_paths, nonv_label = read_nonViolent_videos()

    print("Loading Violent Dataset...")
    violent_cam1_vid_paths, violent_cam2_vid_paths, violent_label = read_violent_videos()

    print("Preparing full dataset...")
    # cam1_video_path = nonv_cam1_vid_paths + violent_cam1_vid_paths
    # cam2_vid_paths = nonv_cam2_vid_paths + violent_cam2_vid_paths
    # labels = nonv_label + violent_label
    # return cam1_video_path, cam2_vid_paths, labels
    non_violent_video_paths = nonv_cam1_vid_paths + nonv_cam2_vid_paths
    violent_video_paths = violent_cam1_vid_paths + violent_cam2_vid_paths
    all_video_paths = non_violent_video_paths + violent_video_paths
    labels = nonv_label + violent_label
    return all_video_paths, labels

## Read the dataset

In [8]:
# cam1_vid_paths, cam2_vid_paths, labels = read_dataset()
all_video_paths, labels = read_dataset()

Loading Non-Violent Dataset...
Loading Violent Dataset...
Preparing full dataset...


In [9]:
# full_dataset = ViolenceDetectionDataset(cam1_vid_paths=cam1_vid_paths, cam2_vid_paths=cam2_vid_paths, labels=labels, normalize=True)
full_dataset = ViolenceDetectionDataset(video_paths=all_video_paths, labels=labels, normalize=True)
ds_len = len(labels)
train_dataset, test_dataset = random_split(
    dataset=full_dataset,
    lengths=[round(ds_len*0.8), round(ds_len*0.2)]
)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=2, shuffle=True, num_workers=0)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=2, shuffle=True, num_workers=0)

## Sample from the DataLoader object

In [10]:
X_batch, y_batch = next(iter(test_dataloader))

In [11]:
X_batch.shape, y_batch.shape

(torch.Size([2, 3, 16, 112, 112]), torch.Size([2, 1]))

## Create model class

### C3D model with FC layers

In [12]:
class ViolenceDetectionModel(nn.Module):
    """
    FC6 C3D model based on the paper "Deep Learning for Automatic Violence Detection: Tests on the AIRTLab Dataset"
    P. Sernani, N. Falcionelli, S. Tomassini, P. Contardo and A. F. Dragoni, "Deep Learning for Automatic Violence Detection: Tests on the AIRTLab Dataset," in IEEE Access, vol. 9, pp. 160580-160595, 2021, doi: 10.1109/ACCESS.2021.3131315.
    """
    def __init__(self):
        super().__init__()
        self.c3d = nn.Sequential(
            nn.Conv3d(in_channels=3, out_channels=64, kernel_size=(3,3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool3d((1,2,2)),
            nn.Conv3d(in_channels=64, out_channels=128, kernel_size=(3,3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool3d((2,2,2)),
            nn.Conv3d(in_channels=128, out_channels=256, kernel_size=(3,3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=256, out_channels=256, kernel_size=(3,3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool3d((2,2,2)),
            nn.Conv3d(in_channels=256, out_channels=512, kernel_size=(3,3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=512, out_channels=512, kernel_size=(3,3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool3d((2,2,2)),
            nn.Conv3d(in_channels=512, out_channels=512, kernel_size=(3,3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(in_channels=512, out_channels=512, kernel_size=(3,3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.ZeroPad3d((0,1,0,1,0,0)), # (padding_left (Width), padding_right (Width), padding_top (Height), padding_bottom (Height), padding_front (Depth), padding_back (Depth))
            nn.MaxPool3d((2,2,2)),
            nn.Flatten(),
            nn.Linear(in_features=8192, out_features=4096), # The in_features here are set for an frame size of 256, if you'll follow the original paper (112 frame size) go with 8192
            nn.ReLU()
        )
        self.ann = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features=4096, out_features=512),
            nn.Dropout(0.5),
            nn.Linear(in_features=512, out_features=1)
        )
    def forward(self, X):
        X = self.c3d(X)
        X = self.ann(X)
        return X

### Metrics Functions

In [13]:
def set_metrics(num_classes:int=None, threshold:float=0.5, task:str="multiclass", device="cpu"):
    """
    Sets metrics from TorchMetrics

    Parameters:
    num_classes (int, Default None): The number of unique classes. Leave it if you have a binary class classification task.
    threshold (float): The threshold to use for a binary class classification task.
    task (str): Whether it's a binary or a multiclass task
    device (torch.device or str): the device to use

    Returns:
    - accuracy_score
    - f1_score
    - precision
    - recall
    """
    if task == "binary":
        accuracy_score = Accuracy(task=task, threshold=threshold).to(device)
        f1_score = F1Score(task=task, threshold=threshold).to(device)
        precision = Precision(task=task, threshold=threshold).to(device)
        recall = Recall(task=task, threshold=threshold).to(device)
    else:
        if num_classes is None:
            raise ValueError("num_classes cannot be `None` for a multiclass classification task")
        accuracy_score = Accuracy(task=task, num_classes=num_classes, average='macro').to(device)
        f1_score = F1Score(task=task, num_classes=num_classes, average='macro').to(device)
        precision = Precision(task=task, num_classes=num_classes, average='macro').to(device)
        recall = Recall(task=task, num_classes=num_classes, average='macro').to(device)
    return accuracy_score, f1_score, precision, recall

def classification_report(num_classes:int, y_true, y_pred, task="multiclass", device="cpu"):
    """
    Creates a full classification report.

    Parameters:
    - num_classes (int): The number of unique classes.
    - y_true (torch.Tensor): The ground truth values.
    - y_pred (torch.Tensor): The predicted values.
    - task (str): binary or multiclass task.
    - device (torch.device or str): the device to use.

    Returns:
    - report (str)
    """
    accuracy_score, f1_score, precision, recall = set_metrics(num_classes, task=task, device=device)
    report_dict = {
        "Accuracy":     accuracy_score(y_pred, y_true).item(),
        "Precision":    precision(y_pred, y_true).item(),
        "Recall":       recall(y_pred, y_true).item(),
        "F1-Score":     f1_score(y_pred, y_true).item()
        }

    report = f"""Full Classification Report:
        - Accuracy:  {report_dict["Accuracy"]}
        - Precision: {report_dict["Precision"]}
        - Recall:    {report_dict["Recall"]}
        - F1-Score:  {report_dict["F1-Score"]}
    """
    return report

### Training functions

In [14]:
task = "binary"

def training_step(loss_fn:torch.nn.modules.loss, optimizer:torch.optim, model:torch.nn.Module, X:torch.Tensor, y:torch.Tensor):
    """
    A single training step, used for testing purposes and inside of bigger training loops functions. made with PyTorch in mind.

    Paramters:
    - loss_fn(torch.nn.modules.loss): The loss function to use
    - optimizer (torch.optim): The optimizer to use
    - model (torch.nn.Module): The model to train
    - X (torch.Tensor): The features matrix
    - y (torch.Tensor): The target variable

    Returns:
    - Loss
    - Logits
    """
    logits = model(X)
    loss = loss_fn(logits, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    return loss.item(), logits

def testing_step(loss_fn:torch.nn.modules.loss, model:torch.nn.Module, X:torch.Tensor, y:torch.Tensor):
    """
    A single inference step, used for testing purposes and inside of bigger testing/inferece loops functions. made with PyTorch in mind.

    Paramters:
    - loss_fn(torch.nn.modules.loss): The loss function to use
    - model (torch.nn.Module): The model to use
    - X (torch.Tensor): The features matrix
    - y (torch.Tensor): The target variable

    Returns:
    - Loss
    - Logits
    """
    with torch.inference_mode():
        logits = model(X)
        loss = loss_fn(logits, y)
    return loss.item(), logits

def test_loop(loss_fn:torch.nn.modules.loss, model:torch.nn.Module, test_dataloader:torch.utils.data.DataLoader, device):
    """
    Inference loop, goes through a full DataLoader object and runs inference on it. Designed with PyTorch in mind

    Parameters:
    - loss_fn(torch.nn.modules.loss): The loss function to use
    - model (torch.nn.Module): The model to use
    - test_dataloader (torch.utils.data.DataLoader): The test dataloader
    - device (torch.device or str): which device to use

    Returns:
    - test_loss: The average testing loss across all batches
    - test_acc: The average testing accuracy across all batches
    - test_f1: The average testing F1 score across all batches
    """
    accuracy_score, f1_score, precision, recall = set_metrics(task=task, device=device)
    test_loss = 0
    model.eval()
    for X_test_batch, y_test_batch in test_dataloader:
        X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)
        sample_test_loss, y_pred = testing_step(
            loss_fn, model, X_test_batch, y_test_batch
        )
        accuracy_score.update(torch.sigmoid(y_pred), y_test_batch)
        f1_score.update(torch.sigmoid(y_pred), y_test_batch)
        test_loss += sample_test_loss
    test_acc = accuracy_score.compute()
    test_f1 = f1_score.compute()
    accuracy_score.reset()
    f1_score.reset()
    test_loss /= len(test_dataloader)
    return test_loss, test_acc, test_f1

def train_loop(loss_fn:torch.nn.modules.loss, optimizer:torch.optim, model:torch.nn.Module, train_dataloader:torch.utils.data.DataLoader, device, lr_scheduler:torch.optim.lr_scheduler=None):
    """
    Training loop, goes through a full DataLoader object and trains on it. Designed with PyTorch in mind

    Parameters:
    - loss_fn(torch.nn.modules.loss): The loss function to use
    - optimizer (torch.optim): The optimizer to use
    - model (torch.nn.Module): The model to train
    - train_dataloader (torch.utils.data.DataLoader): The train dataloader
    - lr_scheduler (torch.optim.lr_scheduler or None): The Learning Rate Scheduler to use
    - device (torch.device or str): which device to use

    Returns:
    - train_loss: The average training loss across all batches
    - train_acc: The average training accuracy across all batches
    - train_f1: The average training F1 score across all batches
    """
    accuracy_score, f1_score, precision, recall = set_metrics(task=task, device=device)
    train_loss = 0
    model.train()
    for X_batch_train, y_batch_train in train_dataloader:
        X_batch_train, y_batch_train = (
            X_batch_train.to(device),
            y_batch_train.to(device),
        )
        sample_train_loss, y_pred = training_step(
            loss_fn, optimizer, model, X_batch_train, y_batch_train
        )
        train_loss += sample_train_loss
        accuracy_score.update(torch.sigmoid(y_pred), y_batch_train)
        f1_score.update(torch.sigmoid(y_pred), y_batch_train)
    if lr_scheduler is not None:
        lr_scheduler.step()
    train_acc = accuracy_score.compute()
    train_f1 = f1_score.compute()
    accuracy_score.reset()
    f1_score.reset()
    train_loss /= len(train_dataloader)
    return train_loss, train_acc, train_f1

def train_test_loop(
    epochs: int, loss_fn:torch.nn.modules.loss, optimizer:torch.optim, model:torch.nn.Module, train_dataloader:torch.utils.data.DataLoader, test_dataloader:torch.utils.data.DataLoader, device, lr_scheduler=None, early_stopping=None, verbose=True,
    ):
    """
    A full train-test loop, where it trains on a given train_dataloader and runs inference using the model on a test_dataloader, for a given number of epochs.

    Parameters:
    - epochs (int): The number of epochs to run
    - loss_fn(torch.nn.modules.loss): The loss function to use
    - optimizer (torch.optim): The optimizer to use
    - model (torch.nn.Module): The model to train
    - train_dataloader (torch.utils.data.DataLoader): The train dataloader
    - test_dataloader (torch.utils.data.DataLoader): The test dataloader
    - device (torch.device or str): the device to use (cuda or cpu)
    - lr_scheduler (torch.optim.lr_scheduler or None): The Learning Rate Scheduler to use
    - early_stopping (custom class or None): The early stopping class to use
    - verbose (bool, default True): Whether to print more information or not

    Returns:
    - None
    """
    model.to(device)
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc, train_f1 = train_loop(
            loss_fn=loss_fn, optimizer=optimizer, model=model, train_dataloader=train_dataloader, lr_scheduler=lr_scheduler, device=device
        )
        test_loss, test_acc, test_f1 = test_loop(loss_fn=loss_fn, model=model, test_dataloader=test_dataloader, device=device)
        print(
            f"""[INFO] Epoch #{epoch+1}\nTraining Loss = {train_loss:.2f} | Testing Loss = {test_loss:.2f}
Training Accuracy = {train_acc:.2%} | Testing Accuracy = {test_acc:.2%}
Training F1-Score = {train_f1:.2%} | Testing F1-Score = {test_f1:.2%}
--------------------------------------------------------"""
        ) if verbose else None

        writer.add_scalars(
            "Train Loss vs Test Loss across all batches",
            {"Train Loss": train_loss, "Test Loss": test_loss},
            epoch + 1,
        )
        writer.add_scalars(
            "Train Accuracy vs Test Accuracy across all batches",
            {"Train Accuracy": train_acc, "Test Accuracy": test_acc},
            epoch + 1,
        )
        if early_stopping is not None:
            early_stopping(test_loss, model)
            if early_stopping.early_stop:
                print("Early stopping")
                writer.close()
                break
    writer.close()
    print("Done Training!")

### Initialize the model

In [15]:
model = ViolenceDetectionModel()
loss_fn = nn.BCEWithLogitsLoss()
lr=3e-3
optimizer = optim.Adam(model.parameters(), lr=lr)

### Train model

In [16]:
model.to(device)
epochs = 5
train_test_loop(
    epochs=epochs,
    loss_fn=loss_fn, optimizer=optimizer, model=model,
    train_dataloader=train_dataloader, test_dataloader=test_dataloader,
    device=device,
    lr_scheduler=None, early_stopping=None,
    verbose=True,
)

  0%|          | 0/5 [00:00<?, ?it/s]

[INFO] Epoch #1
Training Loss = 715.91 | Testing Loss = 0.67
Training Accuracy = 65.00% | Testing Accuracy = 61.43%
Training F1-Score = 78.41% | Testing F1-Score = 76.11%
--------------------------------------------------------
[INFO] Epoch #2
Training Loss = 0.65 | Testing Loss = 0.68
Training Accuracy = 66.07% | Testing Accuracy = 61.43%
Training F1-Score = 79.30% | Testing F1-Score = 76.11%
--------------------------------------------------------
[INFO] Epoch #3
Training Loss = 0.65 | Testing Loss = 0.67
Training Accuracy = 66.07% | Testing Accuracy = 61.43%
Training F1-Score = 79.57% | Testing F1-Score = 76.11%
--------------------------------------------------------
[INFO] Epoch #4
Training Loss = 0.65 | Testing Loss = 0.67
Training Accuracy = 67.14% | Testing Accuracy = 61.43%
Training F1-Score = 80.00% | Testing F1-Score = 76.11%
--------------------------------------------------------
[INFO] Epoch #5
Training Loss = 0.66 | Testing Loss = 0.70
Training Accuracy = 66.07% | Testin